In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv('./kaggle/input/spaceship-titanic/train_Agelow12.csv')

In [5]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [6]:
# 범주형 데이터 변환
def df_replace(dataframe):
    
    global df_train,df
    df_temp = df
    df = dataframe
    df = df.replace({'55 Cancri e':0,'PSO J318.5-22':1,'TRAPPIST-1e':2})
    df = df.replace({'Earth':0,'Europa':1,'Mars':2})
    df = df.replace({False:0,True:1})

    # Cabin : 번호  
    df = df.dropna(subset = ['Cabin'],axis=0)
   
    df['Cabin'] = df['Cabin'].str[0]
    df.insert(4, 'Cabin_num', df_temp['Cabin'].str[2:-2])
    df.insert(5, 'Cabin_port', df_temp['Cabin'].str[-1])
    df = pd.get_dummies(df, columns = ['Cabin'], dummy_na=False)
    # df = df.replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7})
    df = df.replace({'P':0,'S':1})

    col1 = df.columns[-8:].to_list()
    col2 = df.columns[3:-8].to_list()
    col3 = df.columns[:3].to_list()
    df_col = col3 + col1 + col2
    df = df[df_col]

    ## 범주형 데이터 확인할 수 있게 dict로 만들어 놓은 것
    # Cabin = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7}
    Cabin_port = {'P':0, 'S':1} #P : 배의 좌현(port side) S: 배의 우현(starboard side) 
    Destination = {'55 Cancri e': 0, 'PSO J318.5-22': 1,'TRAPPIST-1e':2}
    HomePlanet = {'Earth':0,'Europa':1,'Mars':2}
    CryoSleep = {'False':0,'True':1}
    VIP = {'False':0,'True':1}
    Transported = {'False':0,'True':1}


    df = df.astype({'HomePlanet':'float','CryoSleep':'float','Cabin_num':'float','Cabin_port':'float',
    'Destination':'float','Age':'float','VIP':'float','RoomService':'float','FoodCourt':'float'
    ,'ShoppingMall':'float','Spa':'float','VRDeck':'float','Transported':'float',
    'Cabin_A':'float','Cabin_B':'float','Cabin_C':'float','Cabin_D':'float','Cabin_E':'float','Cabin_F':'float',
    'Cabin_G':'float','Cabin_T':'float',
    })
    

df_replace(df)

df_train = df

In [7]:
df_train.isnull().sum()

PassengerId       0
HomePlanet      195
CryoSleep       210
Cabin_A           0
Cabin_B           0
Cabin_C           0
Cabin_D           0
Cabin_E           0
Cabin_F           0
Cabin_G           0
Cabin_T           0
Cabin_num         0
Cabin_port        0
Destination     178
Age             175
VIP             197
RoomService     158
FoodCourt       166
ShoppingMall    186
Spa             167
VRDeck          160
Name            198
Transported       0
dtype: int64

### null값 처리
XGBOOST를 사용해서 pridct값을 null값으로 대체

In [8]:
df_train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin_A', 'Cabin_B',
       'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_T',
       'Cabin_num', 'Cabin_port', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported'],
      dtype='object')

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import joblib

def fillnullclassifier(column_name):

    global df_update

    df_column_name = df_train[df_train[column_name].isnull()]

    df_train01 = df_train.dropna(subset=[column_name])

    X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId',column_name,'Name','Transported'],axis=1),df_train01[column_name], test_size=0.2, random_state=42)
    model = xgb.XGBClassifier(early_stopping_rounds=10)
    eval_set = [(X_test, y_test)]
    model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

    # 모델 저장
    model_path = './' + 'xgb_model/' +column_name +'.model'
    # model.save_model(model_path)
    joblib.dump(model, open(model_path, 'wb'))

    pred = model.predict(X_test)
    acc = accuracy_score(y_test,pred)
    print(acc)
    
    df_column_name_null = df_column_name.drop(column_name,axis=1)

    pred = model.predict(df_column_name_null.drop(['PassengerId','Name','Transported'],axis=1))

    df_column_name[column_name] = pred

    df_update = pd.concat([df_train01,df_column_name],axis=0)

fillnullclassifier('HomePlanet')

0.9397590361445783


C:\Users\deepBlue\AppData\Local\Temp\ipykernel_1452\86158164.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_column_name[column_name] = pred


In [11]:
df_update[df_update['HomePlanet'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,...,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
